How to parse the dates at the time of loading data  
<code> df = pandas.read_csv('data.csv', parse_dates=['date'])</code>

Filter the data based on the date  
<code>df = df[[d.month == 1 for d in df.date]].copy()</code>  
* fetch only the january month data

club the yearly data into a fraction of day of year from a specific date  
<code>df["years_since_1990"] = [(d.year + d.timetuple().tm_yday / 365.25) - 1990 for d in df.date]</code>

<b>Normalize</b> scale and offset the data column  
<code>df["normalised_temperature"] = (df["min_temperature"] - np.mean(df["min_temperature"])) / np.std(df["min_temperature"])</code>

To show the march $1^{st}$ of 1990  
<code>from datetime import datetime  
print(datetime(1990,3,1)) # yyyy,mm,dd</code>

Filter data based on list of dates  
<code>desired_dates = [  
    datetime(1950,2,1),  
    datetime(1960,2,1),  
    datetime(1970,2,1),  
    datetime(1980,2,1),  
]  
  
df = df[df.date.isin(desired_dates)].copy()</code>